<a href="https://colab.research.google.com/github/AndrewwBC/treinamento-h2ia/blob/main/Buscas_sem_informa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [77]:
import numpy as np
from typing import Literal
import time

class Data_Structure:
  def __init__(self):
    self.items = []

  def is_empty(self):
    return self.items == []

  def clear(self):
    self.items = []

  def push(self, item):
    self.items.append(item)

  def pop_as_a_stack(self):
    return self.items.pop()

  def pop_as_a_queue(self):
    return self.items.pop(0)

  def __str__(self):
    return str(self.items)

class Problem:
    def __init__(self, initial_state):
        self.initial_state = initial_state

class Node:
    def __init__(self, state, parent=None, action=None, depth=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth

    def __str__(self):
        return str(self.state)

    def get_depth(self):
      return self.depth

a = np.arange(0,9)
puzzle = np.array([
    [8, 6, 7],
    [2, 5, 4],
    [3, 0, 1]
])

problem = Problem(puzzle.copy())

target_puzzle = puzzle = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
])

def find_zero(puzzle):
  loc = np.where(puzzle == 0)
  return loc[0][0], loc[1][0]

def actions_by_neighbors(puzzle):
  row, col = find_zero(puzzle)
  neighbors = []

  if row > 0:
    neighbors.append("up")
  if row < 2:
    neighbors.append("down")
  if col > 0:
    neighbors.append("left")
  if col < 2:
    neighbors.append("right")
  return neighbors

def move_zero_to(state, action):
  row, col = find_zero(state)
  aux = state.copy()

  if action == "up":
    aux[row][col] = aux[row - 1][col]
    aux[row - 1][col] = 0

  if action == "down":
    aux[row][col] = aux[row + 1][col]
    aux[row + 1][col] = 0

  if action == "right":
    aux[row][col] = aux[row][col + 1]
    aux[row][col + 1] = 0

  if action == "left":
    aux[row][col] = aux[row][col - 1]
    aux[row][col - 1] = 0

  return aux

def expand(node, state):
  successors = {}

  for action in actions_by_neighbors(state):

    result_of_action = move_zero_to(state, action)

    child = Node(state=result_of_action, parent=node, action=action, depth=node.depth + 1)
    successors[action] = child
  return successors


def freeze(state):
    return tuple(map(tuple, state))

SearchType = Literal["DFS", "BFS", "IDDFS"]

class Search:
    def __init__(self):
        self._fringe = Data_Structure()
        self._fringe.push(Node(state=problem.initial_state))
        self._visited = set()
        self._max_depth = -1
        self._node = Node(state=problem.initial_state)

    def _reset(self):
        self._fringe.clear()
        self._fringe.push(Node(state=problem.initial_state))
        self._max_depth = -1
        self._clear_visited()

    def _report(self, is_failure):
        node = self._node
        depth = self._max_depth
        message = "Fail" if is_failure else "Success"
        self._reset()
        return {"node": node, "message": message, "depth": depth}

    def _clear_visited(self):
        self._visited.clear()

    def _add_visited(self, state):
        self._visited.add(state)

    def DFS(self):
        while True:
            if self._fringe.is_empty():
                self._clear_visited()
                return self._report(True)

            node = self._fringe.pop_as_a_stack()
            self._node = node

            if node.depth > self._max_depth:
                self._max_depth = node.depth

            state_key = freeze(node.state)
            if state_key in self._visited:
                continue
            self._visited.add(state_key)

            if np.array_equal(node.state, target_puzzle):
                self._clear_visited()
                return self._report(False)

            for child in expand(node, node.state).values():
                if freeze(child.state) not in self._visited:
                    self._fringe.push(child)

    def BFS(self):
        while True:
            if self._fringe.is_empty():
                self._clear_visited()
                return self._report(True)

            node = self._fringe.pop_as_a_queue()
            self._node = node

            if node.depth > self._max_depth:
                self._max_depth = node.depth

            state_key = freeze(node.state)
            if state_key in self._visited:
                continue
            self._visited.add(state_key)

            if np.array_equal(node.state, target_puzzle):
                self._clear_visited()
                return self._report(False)

            for child in expand(node, node.state).values():
                if freeze(child.state) not in self._visited:
                    self._fringe.push(child)

    def IDDFS(self, depth):
        while True:
              if self._fringe.is_empty():
                  self._clear_visited()
                  return self._report(True)

              node = self._fringe.pop_as_a_stack()
              self._node = node

              if node.depth > self._max_depth:
                self._max_depth = node.depth

              state_key = freeze(node.state)
              if state_key in self._visited:
                  continue
              self._visited.add(state_key)

              if np.array_equal(node.state, target_puzzle):
                    self._clear_visited()
                    return self._report(False)
              else:
                if node.depth == depth:
                    continue
                for child in expand(node, node.state).values():
                  if freeze(child.state) not in self._visited:
                      self._fringe.push(child)


search = Search()

def call_and_temporize(search_method, has_limit = False):
  init = time.time()
  result = {}

  if has_limit:
    limit = 100
    initial_limit = 0
    while initial_limit < limit:
      result = search_method(initial_limit)
      if result["message"] == "Success":
        break
      initial_limit += 1
  else:
    result = search_method()
  end = time.time()

  return {
      "time": round(end - init, 4),
      "depth": result.get("depth")
      }

DFS = call_and_temporize(search.DFS)
BFS = call_and_temporize(search.BFS)
IDDFS = call_and_temporize(search.IDDFS, True)

print("\n")
print(100*"-")
print("Tempo (Segundos) -- Profundidade \n")
print("DFS: ", DFS["time"], "-- ", DFS["depth"])
print("BFS: ", BFS["time"], "-- ", BFS["depth"])
print("ID-DFS: ", IDDFS["time"], "-- ", IDDFS["depth"])

A saída de streaming foi truncada nas últimas 5000 linhas.
(array([1]), array([0]))
(array([1]), array([0]))
(array([1]), array([0]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([0]))
(array([1]), array([0]))
(array([1]), array([0]))
(array([1]), array([0]))
(array([2]), array([1]))
(array([2]), array([1]))
(array([2]), array([1]))
(array([2]), array([1]))
(array([1]), array([0]))
(array([1]), array([0]))
(array([1]), array([0]))
(array([1]), array([0]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([0]))
(array([1]), array([0]))
(array([1]), array([0]))
(array([1]), array([0]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), array([2]))
(array([1]), arr

KeyboardInterrupt: 

## Discorra sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento


 1 - DFS

 A Busca em profundidade possui o menor consumo de memória, pois a memória necessária está atrelada a profundidade da árvore e as ramificações de um ramo apenas. Então, o número médio de ações de cada nó multiplicado pela profundidade gera a complexidade de memória O(número médio de ações * profundidade máxima).

 2 - BFS

Em contrapartida, BFS irá enfileirar todas as ações possíveis a partir de cada nó. Então se cada nó possuir uma média de duas ações, significa que o nó inicial terá dois filhos, quatro netos, oito bisnetos, etc. Ou seja, a complexidade de memória é exponencial: O(número médio de ações^profundidade da solução).

3 - IDDFS

Esta estratégia de busca visa combinar as vantagens de ambos algoritmos anteriormente citados. Apesar de usar um "DFS", ele explora com profundidade crescente, ou seja, pode encontrar a solução mais rapidamente sem explorar todo espaço de possibilidades desnecessariamente, ao mesmo tempo que não consome toda memória que o BFS requer. A complexidade de memória é a mesma do DFS, por óbvio.

